In [43]:
!pip install python-dateutil
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta
import sklearn as sk
import json

# Reading the file
with open("Enter text file", 'r') as file:
    destinations = file.read().splitlines()
API_key = 'Enter API key'

In [44]:
# Creating the new DataFrame
data = {
    'Target': [],
    'Distance_km': [],
    'Duration (hour+minutes)': [],
    'Longitude': [],
    'Latitude': []
}

## Latitude and Longitude


In [45]:
# Google Geocode function for data retrieving
def get_geocode_data(destination):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={destination}&key=%s'%(API_key)
    # Send a GET request to the API and parse the response as JSON
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error")
        else:
            try:
                response_data = response.json()
            except: 
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    # Extract the relevant information from the response JSON
    result = response_data['results'][0]
    lat = result['geometry']['location']['lat'] # Extract the latitude
    lng = result['geometry']['location']['lng'] # Extract the longitude
    return lat, lng


## Distance and Duration


In [46]:
# Google Distance Matrix for distance and duration
# Function to retrieve distance and duration information from Google Distance Matrix service
def get_distance_data(destination):
    origin = 'Tel Aviv, Israel'
    # Construct the URL for the API request with the destination and API key
    url = f'https://maps.googleapis.com/maps/api/distancematrix/json?units=metric&origins={origin}&destinations={destination}&key=%s'%(API_key)
    try:
        # Send a GET request to the API and parse the response as JSON
        response = requests.get(url)
        if not response.status_code == 200:
            print ("HTTP error")
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    # Extract the relevant information from the response JSON
    element = response_data['rows'][0]['elements'][0]
    distance = element['distance']['text']  # Extract the distance as text
    duration_in_minutes = element['duration']['value']//60 # converting time to minutes only
    duration = "%s hours %s minutes" % (duration_in_minutes//60, duration_in_minutes%60) # dividing the duration to hours and minutes
    return(distance, duration)



In [47]:
# Storing all the information about the destinations in DataFrame 
for destination in destinations:
    distance, duration = get_distance_data(destination)
    lat, lng = get_geocode_data(destination)
    data['Target'].append(destination)
    data['Distance_km'].append(distance)
    data['Duration (hour+minutes)'].append(duration)
    data['Longitude'].append(lng)
    data['Latitude'].append(lat)
    
df = pd.DataFrame(data)


## DataFrame content

In [48]:
df

,Target,Distance_km,Duration (hour+minutes),Longitude,Latitude
0,Istanbul,"1,815 km",21 hours 2 minutes,28.978359,41.008238
1,Amsterdam,"4,533 km",48 hours 3 minutes,4.904139,52.367573
2,Valletta,"3,793 km",50 hours 52 minutes,14.514100,35.899237
3,Basel,"4,093 km",44 hours 2 minutes,7.588576,47.559599
4,Doha,"2,164 km",22 hours 38 minutes,51.531040,25.285447


## 3 cities furthest from Tel Aviv

In [49]:
# Sort the dataframe by distance in descending order
df_sorted = df.sort_values('Distance_km', ascending=False)

# Get the top 3 cities furthest from Tel Aviv
furthest_cities = df_sorted.head(3)

# Print the result
furthest_cities

,Target,Distance_km,Duration (hour+minutes),Longitude,Latitude
1,Amsterdam,"4,533 km",48 hours 3 minutes,4.904139,52.367573
3,Basel,"4,093 km",44 hours 2 minutes,7.588576,47.559599
2,Valletta,"3,793 km",50 hours 52 minutes,14.514100,35.899237
